In [2]:
import utils.image_analysis_utils as iau
import numpy as np
import utils.enhanced_net_detection as end
import utils.image_analysis_utils as iau

In [ ]:
# 🌊 SONAR DATA LOADING & NET DETECTION

# CONFIGURATION PARAMETERS
# =========================
# Modify these parameters to customize the analysis

# NPZ file selection (modify to select different file)
NPZ_FILE = None  # Auto-select first available file
# NPZ_FILE = "2024-08-20_13-39-34_cones.npz"  # Or specify a specific file

# Detection parameters
DETECTION_PARAMS = {
    'blur_ksize': 70,           # Main blur kernel size
    'blur_sigma': 10.0,         # Main blur sigma
    'thr_percentile': 80,       # Percentile threshold for segmentation
    'min_area_px': 200,         # Minimum area for blob detection
    'pre_blur_ksize': 15,       # Pre-blur kernel size
    'pre_blur_sigma': 3.0,      # Pre-blur sigma  
    'strong_blur_ksize': 25,    # Strong initial blur kernel size
    'strong_blur_sigma': 5.0    # Strong initial blur sigma
}

# Blob analysis comparison parameters
BLOB_PARAMS = {
    'blur_ksize': 70, 
    'blur_sigma': 10.0,
    'thr_percentile': 80, 
    'min_area_px': 80,
    'open_close': 1,
    'save_csv': "exports/outputs/largest_blob.csv",
    'save_mp4': "exports/outputs/largest_blob.mp4",
    'fps': 15, 
    'progress': True
}

# LOAD DATA
# =========
print("🔍 Loading sonar data for enhanced net detection...")

# Select and load NPZ file
NPZ = end.select_npz_file(NPZ_FILE)
print(f"Loading: {NPZ}")

try:
    # Load cone data
    cones, ts, extent, meta = iau.load_cone_run_npz(NPZ)
    
    print(f"✅ Loaded {len(cones)} frames")
    print(f"   Frame shape: {cones[0].shape}")
    print(f"   Data type: {cones.dtype}")
    print(f"   Extent: {extent}")
    print(f"   Time range: {ts[0]} to {ts[-1]}")
    
    # Check for NaN values
    frame_sample = cones[0]
    has_nan = iau.np.isnan(frame_sample).any()
    print(f"   Contains NaN: {has_nan}")
    
    if has_nan:
        nan_count = iau.np.isnan(frame_sample).sum()
        total_pixels = frame_sample.size
        print(f"   NaN pixels: {nan_count}/{total_pixels} ({nan_count/total_pixels*100:.1f}%)")
    
    print(f"   Value range: {iau.np.nanmin(frame_sample):.3f} to {iau.np.nanmax(frame_sample):.3f}")
    
    print("\n🎯 Cone data loaded and ready for enhanced net detection!")
    
except Exception as e:
    print(f"❌ Error loading NPZ file: {e}")
    print(f"   File path: {NPZ}")
    print(f"   Make sure the file exists and was created correctly")
    raise

# RUN COMPARISON BLOB ANALYSIS
# ============================
df = end.run_comparison_blob_analysis(NPZ, BLOB_PARAMS)

if df is not None and len(df) > 0:
    print(f"\n📋 First few blob detections:")
    display(df.head())

🔍 Loading sonar data for enhanced net detection...
🔍 Found 5 NPZ files:
   1. 2024-08-22_14-06-43_data_cones.npz
   2. 2024-08-20_13-40-35_data_cones.npz
   3. 2024-08-22_14-47-39_data_cones.npz
   4. 2024-08-22_14-29-05_data_cones.npz
   5. 2024-08-20_13-39-34_data_cones.npz

✅ Auto-selected: 2024-08-22_14-06-43_data_cones.npz
Loading: exports/outputs/2024-08-22_14-06-43_data_cones.npz
✅ Loaded 592 frames
   Frame shape: (700, 900)
   Data type: float32
   Extent: (-8.66025447845459, 8.66025447845459, 0.0, 10.0)
   Time range: 2024-08-22 12:07:23.912630+00:00 to 2024-08-22 12:07:23.912630+00:00
   Contains NaN: True
   NaN pixels: 250080/630000 (39.7%)
   Value range: 0.000 to 1.000

🎯 Cone data loaded and ready for enhanced net detection!

📊 Running largest blob analysis for comparison...
✅ Loaded 592 frames
   Frame shape: (700, 900)
   Data type: float32
   Extent: (-8.66025447845459, 8.66025447845459, 0.0, 10.0)
   Time range: 2024-08-22 12:07:23.912630+00:00 to 2024-08-22 12:07:2

,t_idx,ts,cx_px,cy_px,x_m,y_m,x0,y0,x1,y1,area_px,method
0,0,2024-08-22 12:07:23.912630+00:00,466.521744,89.647613,0.327948,1.282512,342,68,584,121,8922,classical_largest_blob
1,1,2024-08-22 12:07:23.912630+00:00,458.917171,87.723063,0.181435,1.254979,342,63,571,117,9912,classical_largest_blob
2,2,2024-08-22 12:07:23.912630+00:00,454.587106,87.562319,0.098010,1.252680,325,61,568,114,10005,classical_largest_blob
3,3,2024-08-22 12:07:23.912630+00:00,451.097422,87.429978,0.030777,1.250787,323,60,564,115,9854,classical_largest_blob
4,4,2024-08-22 12:07:23.912630+00:00,445.241549,87.485906,-0.082045,1.251587,321,58,562,117,9969,classical_largest_blob


In [3]:
# 🧪 TEST ENHANCED NET DETECTION
# ===============================

# TEST PARAMETERS
# ===============
TEST_FRAMES = 2  # Number of frames to test
VERBOSE = True   # Show detailed output
NPZ = "exports/outputs/2024-08-20_13-39-34_data_cones.npz"

DETECTION_PARAMS = {
    'blur_ksize': 70,           # Main blur kernel size
    'blur_sigma': 10.0,         # Main blur sigma
    'thr_percentile': 80,       # Percentile threshold for segmentation
    'min_area_px': 200,         # Minimum area for blob detection
    'pre_blur_ksize': 15,       # Pre-blur kernel size
    'pre_blur_sigma': 3.0,      # Pre-blur sigma  
    'strong_blur_ksize': 25,    # Strong initial blur kernel size
    'strong_blur_sigma': 5.0    # Strong initial blur sigma
}

# Reset tracking for testing
end.net_tracker.reset()

print("🧪 Testing enhanced net detection with tracking...")

try:
    # Load cone data
    cones, ts, extent, meta = iau.load_cone_run_npz(NPZ)
    
    print(f"✅ Loaded {len(cones)} frames")
    print(f"   Frame shape: {cones[0].shape}")
    print(f"   Data type: {cones.dtype}")
    print(f"   Extent: {extent}")
    print(f"   Time range: {ts[0]} to {ts[-1]}")
    
    # Check for NaN values
    frame_sample = cones[0]
    has_nan = iau.np.isnan(frame_sample).any()
    print(f"   Contains NaN: {has_nan}")
    
    if has_nan:
        nan_count = iau.np.isnan(frame_sample).sum()
        total_pixels = frame_sample.size
        print(f"   NaN pixels: {nan_count}/{total_pixels} ({nan_count/total_pixels*100:.1f}%)")
    
    print(f"   Value range: {iau.np.nanmin(frame_sample):.3f} to {iau.np.nanmax(frame_sample):.3f}")
    
    print("\n🎯 Cone data loaded and ready for enhanced net detection!")

except Exception as e:
    print(f"❌ Error loading NPZ file: {e}")
    print(f"   File path: {NPZ}")
    print(f"   Make sure the file exists and was created correctly")
    raise

# Run test on sample frames
test_results = end.test_enhanced_detection(
    cones=cones,
    detection_params={**DETECTION_PARAMS, 'verbose': VERBOSE},
    test_frames=TEST_FRAMES
)

print(f"\n✅ Testing complete! Processed {len(test_results)} frames")

# Show summary of results
for i, result in enumerate(test_results):
    print(f"\nFrame {i}:")
    print(f"   Net detected: {'✅' if result['net_detected'] else '❌'}")
    if result['tracking_info']['position']:
        pos = result['tracking_info']['position']
        print(f"   Position: ({pos[0]:.1f}, {pos[1]:.1f})")
        print(f"   Stable frames: {result['tracking_info']['stable_frames']}")
        if result['net_info']:
            print(f"   Confidence: {result['net_info']['confidence']:.2f}")

print(f"\n🎯 Enhanced detection system is working correctly!")
print(f"🔧 To adjust detection sensitivity, modify DETECTION_PARAMS in Cell 2")

🧪 Testing enhanced net detection with tracking...
✅ Loaded 699 frames
   Frame shape: (700, 900)
   Data type: float32
   Extent: (-8.66025447845459, 8.66025447845459, 0.0, 10.0)
   Time range: 2024-08-20 11:40:23.268609+00:00 to 2024-08-20 11:40:23.268609+00:00
   Contains NaN: True
   NaN pixels: 250080/630000 (39.7%)
   Value range: 0.000 to 1.000

🎯 Cone data loaded and ready for enhanced net detection!
🧪 Testing enhanced net detection with tracking...
Using stronger blurring and parallel line analysis

📍 Testing on frame 0 (shape: (700, 900))...

🔍 Enhanced Net Detection:
🔍 Running enhanced net detection with tight tracking...
   Total lines detected: 55
   Parallel pairs found: 39
   Net detected: ✅ YES

🎯 NET DETAILS:
   Position: (481.5, 337.2)
   Orientation: 167.9°
   Width: 15.4 pixels
   Confidence: 10.89
   Line 1 length: 65.5 pixels
   Line 2 length: 102.2 pixels
   Contour area: 622 pixels

📊 ALL PARALLEL PAIRS:
   Pair 1: width=15.4px, angle=167.9°, confidence=10.89
   

In [4]:
# 🎬 CREATE ENHANCED NET DETECTION VIDEO
# ======================================

# VIDEO PARAMETERS
# ================
VIDEO_PARAMS = {
    'output_filename': 'enhanced_net_detection.mp4',
    'fps': 15,
    'max_frames': 1000,  # Set to None for all frames
    'detection_params': DETECTION_PARAMS
}

# NPZ FILE FOR VIDEO
# ==================
# You can specify a different NPZ file for video creation if desired
VIDEO_NPZ = NPZ  # Use the same NPZ file as loaded above
# VIDEO_NPZ = end.select_npz_file("2024-08-20_13-39-34_cones.npz")  # Or select different file



# Create enhanced net detection video
enhanced_video = end.create_enhanced_detection_video(
    npz_path=VIDEO_NPZ,
    **VIDEO_PARAMS
)

🎬 Creating enhanced net detection video: exports/outputs/enhanced_net_detection.mp4
   Processing 699 frames at 15 FPS
   Processing frame 0/699...
   Processing 699 frames at 15 FPS
   Processing frame 0/699...
   Processing frame 50/699...
   Processing frame 50/699...
   Processing frame 100/699...
   Processing frame 100/699...
   Processing frame 150/699...
   Processing frame 150/699...
   Processing frame 200/699...
   Processing frame 200/699...
   Processing frame 250/699...
   Processing frame 250/699...
   Processing frame 300/699...
   Processing frame 300/699...
   Processing frame 350/699...
   Processing frame 350/699...
   Processing frame 400/699...
   Processing frame 400/699...
   Processing frame 450/699...
   Processing frame 450/699...
   Processing frame 500/699...
   Processing frame 500/699...
   Processing frame 550/699...
   Processing frame 550/699...
   Processing frame 600/699...
   Processing frame 600/699...
   Processing frame 650/699...
   Processing f

In [ ]:
# 📋 CONFIGURATION GUIDE
# ======================

print("⚙️  Enhanced Net Detection Configuration Guide")
print("=" * 50)

print("\n🔧 DETECTION_PARAMS (Cell 2):")
print("   blur_ksize: Main blur kernel size (larger = more blur)")
print("   blur_sigma: Main blur strength (higher = smoother)")
print("   thr_percentile: Segmentation threshold (80-90 typical)")
print("   min_area_px: Minimum blob area (100-300 typical)")
print("   pre_blur_ksize: Pre-processing blur size")
print("   strong_blur_ksize: Initial noise reduction blur")

print("\n🎬 VIDEO_PARAMS (Cell 5):")
print("   output_filename: Name of output video file")
print("   fps: Video frame rate (10-30 typical)")
print("   max_frames: Maximum frames to process (None = all)")

print("\n📊 BLOB_PARAMS (Cell 2):")
print("   Standard blob analysis parameters for comparison")

print("\n🎯 TRACKING CONFIGURATION:")
print("   These are set in the utility file (utils/enhanced_net_detection.py):")
print("   • roi_padding: 80px (search area around last position)")
print("   • max_jump_distance: 30px (max movement per frame)")
print("   • search_radius: 50px (visualization circle)")
print("   • Position smoothing: 70% new, 30% old")

print("\n💡 TUNING TIPS:")
print("   • If too many false detections: increase thr_percentile")
print("   • If missing detections: decrease thr_percentile, increase blur")
print("   • If tracking jumps around: decrease max_jump_distance")
print("   • If tracking too slow: increase roi_padding")
print("   • For noisy data: increase strong_blur_ksize")

print("\n🔍 TO MODIFY TRACKING PARAMETERS:")
print("   Edit the NetTracker.__init__() method in:")
print("   utils/enhanced_net_detection.py")

print("\n📁 FILE SELECTION:")
print("   • Set NPZ_FILE = None for auto-selection")
print("   • Set NPZ_FILE = 'filename.npz' for specific file")
print("   • Set VIDEO_NPZ to use different file for video")

In [ ]:
# 🚀 QUICK USAGE EXAMPLES
# =======================

print("🚀 Quick Usage Examples")
print("=" * 30)

print("\n📝 Example 1: Test single frame detection")
print("# Reset tracking and test on frame 0")
print("end.net_tracker.reset()")
print("result = end.detect_net_with_tracking(cones[0], **DETECTION_PARAMS)")
print("print(f'Net detected: {result[\"net_detected\"]}')")

print("\n📝 Example 2: Create custom video")
print("# Create video with custom parameters")
print("custom_params = {")
print("    'blur_ksize': 100,    # Stronger blur")
print("    'thr_percentile': 85, # Higher threshold")
print("    'min_area_px': 150    # Smaller minimum area")
print("}")
print("video_path = end.create_enhanced_detection_video(")
print("    NPZ, 'custom_detection.mp4', fps=20, max_frames=500,")
print("    detection_params=custom_params)")

print("\n📝 Example 3: Process different NPZ file")
print("# Select and process a different file")
print("other_npz = end.select_npz_file('2024-08-22_14-06-43_cones.npz')")
print("other_cones, _, _, _ = iau.load_cone_run_npz(other_npz)")
print("end.net_tracker.reset()  # Important: reset tracking!")
print("test_results = end.test_enhanced_detection(other_cones, DETECTION_PARAMS)")

print("\n📝 Example 4: Access tracking information")
print("# Get current tracking state")
print("print(f'Last position: {end.net_tracker.last_position}')")
print("print(f'Stable frames: {end.net_tracker.stable_frames}')")
print("print(f'Confidence history: {end.net_tracker.confidence_history}')")

print("\n📝 Example 5: Manually reset tracking")
print("# Reset tracking state (useful when switching data)")
print("end.net_tracker.reset()")
print("print('Tracking reset - ready for new analysis')")

print("\n💡 Remember:")
print("   • Always reset tracking when switching to new data")
print("   • Modify DETECTION_PARAMS in Cell 2 for global changes")
print("   • Check utils/enhanced_net_detection.py for advanced customization")